# Evaluation of MLP log-likelihood

Confirm PyTorch and manually coded MLP log-likelihood coincide

In [1]:
## Import packages

import torch
import torch.nn as nn
import torch.nn.functional as F

from eeyore.models.mlp import Hyperparameters, MLP
from eeyore.data import XOR

## Compute MLP log-likelihood using eeyore API version

In [2]:
### Load XOR data

xor = XOR()
data = xor.data
labels = xor.labels

In [3]:
### Setup MLP model

hparams = Hyperparameters([2, 2, 1])
model = MLP(hparams=hparams, dtype=torch.float64)
out = model(data)

In [4]:
### Compute MLP log-likelihood using eeyore API version

result01 = model.log_lik(data, labels)
result01

tensor(-2.8585, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood using Pytorch lognn.Loss class

In [5]:
### Define logit loss

criterion = nn.BCEWithLogitsLoss(reduction='sum')
loss = criterion(out, labels)

In [6]:
## Define logit function

def logit(p):
    return torch.log(p/(1-p))

In [7]:
### Compute MLP log-likelihood using Pytorch binary_cross_entropy_with_logits

result02a = -F.binary_cross_entropy_with_logits(logit(out), labels, reduction='sum')
result02a

tensor(-2.8585, dtype=torch.float64, grad_fn=<NegBackward>)

In [8]:
### Compute MLP log-likelihood using Pytorch binary_cross_entropy

result02b = -F.binary_cross_entropy(out, labels, reduction='sum')
result02b

tensor(-2.8585, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood manually

In [9]:
### Define sigmoid function

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [10]:
### Define MLP log-likelihood

def loglik(x, y):
    term = y * torch.log(sigmoid(x)) + (1-y) * torch.log(1-sigmoid(x))
    return torch.sum(term)

In [11]:
### Compute MLP log-likelihood manually

result03 = loglik(logit(out), labels)
result03

tensor(-2.8585, dtype=torch.float64, grad_fn=<SumBackward0>)

In [12]:
[p.data.item() for p in [result01, result02a, result02b, result03]]

[-2.8585010367224246,
 -2.8585010367224246,
 -2.8585010367224246,
 -2.858501036722425]